# Описание проекта

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

# 1. Загрузка и подготовка данных

загружаем необходимые для работы библиотеки

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
import seaborn as sns
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from numpy.random import RandomState
import numpy as np


загружаем датасеты

In [2]:
geo_1 = pd.read_csv('/datasets/geo_data_0.csv')
geo_2 = pd.read_csv('/datasets/geo_data_1.csv')
geo_3 = pd.read_csv('/datasets/geo_data_2.csv')

первичный осмотр данных

In [3]:
display(geo_1.head())

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [4]:
display(geo_2.head())

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [5]:
display(geo_3.head())

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


**Вывод**   
Мы можем видеть, что у нас всего 3 призака f0,f1,f2, которые информативны для нахождения целевого признака product.
колонка id не информативна для модели, так как должна являться уникальной, поэтому в дальнейшем мы её удалим

Проверим датасеты на пропуски и корректность формата данных.

In [6]:
geo_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [7]:
geo_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [8]:
geo_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


Пропусков нет, все форматы данных корректны.

#### Проверка на дубликаты.

Найдём дубликаты в первой зоне.

In [9]:
geo_1['id'].duplicated().sum()


10

Найдено 10 дубликатов, посмотрим как они выглядят в таблице

In [10]:
geo_1[geo_1['id'].duplicated()==True]

,id,f0,f1,f2,product
7530,HZww2,1.061194,-0.373969,10.430210,158.828695
41724,bxg6G,-0.823752,0.546319,3.630479,93.007798
51970,A5aEY,-0.180335,0.935548,-2.094773,33.020205
63593,QcMuo,0.635635,-0.473422,0.862670,64.578675
66136,74z30,1.084962,-0.312358,6.990771,127.643327
69163,AGS9W,-0.933795,0.116194,-3.655896,19.230453
75715,Tdehs,0.112079,0.430296,3.218993,60.964018
90815,fiKDv,0.049883,0.841313,6.394613,137.346586
92341,TtcGQ,0.110711,1.022689,0.911381,101.318008
97785,bsk9y,0.378429,0.005837,0.160827,160.637302


ВОзьмём отдельно взятый дубликат

In [11]:
geo_1[geo_1['id']=='HZww2']

,id,f0,f1,f2,product
931,HZww2,0.755284,0.368511,1.863211,30.681774
7530,HZww2,1.061194,-0.373969,10.430210,158.828695


Проведём аналогичные шаги для других датасетов

In [12]:
geo_2['id'].duplicated().sum()


4

In [13]:
geo_2[geo_2['id'].duplicated()==True]

,id,f0,f1,f2,product
41906,LHZR0,-8.989672,-4.286607,2.009139,57.085625
82178,bfPNe,-6.202799,-4.820045,2.995107,84.038886
82873,wt4Uk,10.259972,-9.376355,4.994297,134.766305
84461,5ltQ6,18.213839,2.191999,3.993869,107.813044


In [14]:
geo_2[geo_2['id']=='5ltQ6']

,id,f0,f1,f2,product
5849,5ltQ6,-3.435401,-12.296043,1.999796,57.085625
84461,5ltQ6,18.213839,2.191999,3.993869,107.813044


In [15]:
geo_3['id'].duplicated().sum()

4

In [16]:
geo_3[geo_3['id'].duplicated()==True]

,id,f0,f1,f2,product
43233,xCHr8,-0.847066,2.101796,5.597130,184.388641
49564,VF7Jo,-0.883115,0.560537,0.723601,136.233420
55967,KUPhW,1.211150,3.176408,5.543540,132.831802
95090,Vcm5J,2.587702,1.986875,2.482245,92.327572


In [17]:
geo_3[geo_3['id']=='KUPhW']

,id,f0,f1,f2,product
45404,KUPhW,0.231846,-1.698941,4.990775,11.716299
55967,KUPhW,1.211150,3.176408,5.543540,132.831802


**Вывод:**
Данные совпадают только по id ( признак который не информативен для обучения модели), остальны столбцы не совпадают. Поэтому считаю, что эти дубликаты можно оставить.

#### удаляем лишний столбец, который не информативен для обучения моделей.

In [18]:
good_geo_1 =geo_1.drop(['id'], axis=1)
good_geo_2 =geo_2.drop(['id'], axis=1)
good_geo_3 =geo_3.drop(['id'], axis=1)

**Вывод**
предобработка данных завершена.
заполнение пропусков и удаление дубликатов не требуется. Ненужный столбец удалён

# 2. Обучение и проверка модели

Разобьем данные на обучающую и валидационную выборку в пропорции 75:25

In [19]:
good_geo_1_train, good_geo_1_valid = train_test_split(good_geo_1,  test_size=0.25, random_state=12345)

In [20]:
good_geo_2_train, good_geo_2_valid = train_test_split(good_geo_2,  test_size=0.25, random_state=12345)

In [21]:
good_geo_3_train, good_geo_3_valid = train_test_split(good_geo_3,  test_size=0.25, random_state=12345)

создаём таблицы с признаками и целевым признаком для каждой зоны

In [22]:
features_good_geo_1_train = good_geo_1_train.drop('product', axis=1)
target_good_geo_1_train = good_geo_1_train['product']
features_good_geo_2_train = good_geo_2_train.drop('product', axis=1)
target_good_geo_2_train = good_geo_2_train['product']
features_good_geo_3_train = good_geo_3_train.drop('product', axis=1)
target_good_geo_3_train = good_geo_3_train['product']

features_good_geo_1_valid = good_geo_1_valid.drop('product', axis=1)
target_good_geo_1_valid = good_geo_1_valid['product']
features_good_geo_2_valid = good_geo_2_valid.drop('product', axis=1)
target_good_geo_2_valid = good_geo_2_valid['product']
features_good_geo_3_valid = good_geo_3_valid.drop('product', axis=1)
target_good_geo_3_valid = good_geo_3_valid['product']

Задаем параметры для GridSearchCV, чтобы найти идеальные параметры для модели.

In [23]:
param_grid_logres = {'fit_intercept':['True','False'],
                    'normalize':['True','False'],
                    'copy_X':['True','False'],
                    'n_jobs':[n for n in range(-1,10,1)]}

In [24]:
True, False

(True, False)

Загружаем GsCV

In [25]:
gs_logres_1 = GridSearchCV(LinearRegression(), param_grid = param_grid_logres)
%time

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 8.58 µs


Обучаем модель

In [26]:
gs_logres_1.fit(features_good_geo_1_train, target_good_geo_1_train)

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=LinearRegression(copy_X=True, fit_intercept=True,
                                        n_jobs=None, normalize=False),
             iid='warn', n_jobs=None,
             param_grid={'copy_X': ['True', 'False'],
                         'fit_intercept': ['True', 'False'],
                         'n_jobs': [-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                         'normalize': ['True', 'False']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

Выводим лучшие параметры

In [27]:
gs_logres_1.best_params_

{'copy_X': 'True', 'fit_intercept': 'True', 'n_jobs': -1, 'normalize': 'True'}

Создаем модель с лучшими параметрами

In [28]:
model_1 = LinearRegression(copy_X=True, 
                            fit_intercept=True, 
                            n_jobs=-1, 
                            normalize=True)

Обучаем модель

In [29]:
model_1.fit(features_good_geo_1_train, target_good_geo_1_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=True)

Делаем предсказание

In [30]:
model_1_predict = pd.Series(model_1.predict(features_good_geo_1_valid))

Находим стреднее значение

In [31]:
mean_geo_1 = model_1_predict.mean()

In [32]:
display(mean_geo_1)

92.59256778438035

находим rmse для модели

In [33]:
mse_1 = mean_squared_error(target_good_geo_1_valid,model_1_predict)
rmse_1 = mse_1**0.5
print(rmse_1)

37.5794217150813


Аналогично для других датасетов

In [34]:
gs_logres_2 = GridSearchCV(LinearRegression(), param_grid = param_grid_logres)
%time

CPU times: user 2 µs, sys: 3 µs, total: 5 µs
Wall time: 8.82 µs


In [35]:
gs_logres_2.fit(features_good_geo_2_train, target_good_geo_2_train)

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=LinearRegression(copy_X=True, fit_intercept=True,
                                        n_jobs=None, normalize=False),
             iid='warn', n_jobs=None,
             param_grid={'copy_X': ['True', 'False'],
                         'fit_intercept': ['True', 'False'],
                         'n_jobs': [-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                         'normalize': ['True', 'False']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [36]:
gs_logres_2.best_params_

{'copy_X': 'True', 'fit_intercept': 'True', 'n_jobs': -1, 'normalize': 'True'}

In [37]:
model_2 = LinearRegression(copy_X=True, 
                            fit_intercept=True, 
                            n_jobs=-1, 
                            normalize=True)

In [38]:
model_2.fit(features_good_geo_2_train, target_good_geo_2_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=True)

In [39]:
model_2_predict = pd.Series(model_2.predict(features_good_geo_2_valid))

In [40]:
mean_geo_2 = model_2_predict.mean()

In [41]:
display(mean_geo_2)

68.72854689544602

In [42]:
mse_2 = mean_squared_error(target_good_geo_2_valid,model_2_predict)
rmse_2 = mse_2**0.5
print(rmse_2)

0.8930992867756158


In [43]:
gs_logres_3 = GridSearchCV(LinearRegression(), param_grid = param_grid_logres)


In [44]:
gs_logres_3.fit(features_good_geo_3_train, target_good_geo_3_train)

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=LinearRegression(copy_X=True, fit_intercept=True,
                                        n_jobs=None, normalize=False),
             iid='warn', n_jobs=None,
             param_grid={'copy_X': ['True', 'False'],
                         'fit_intercept': ['True', 'False'],
                         'n_jobs': [-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                         'normalize': ['True', 'False']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [45]:
gs_logres_3.best_params_

{'copy_X': 'True', 'fit_intercept': 'True', 'n_jobs': -1, 'normalize': 'True'}

In [46]:
model_3 = LinearRegression(copy_X=True, 
                            fit_intercept=True, 
                            n_jobs=-1, 
                            normalize=True)

In [47]:
model_3.fit(features_good_geo_3_train, target_good_geo_3_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=True)

In [48]:
model_3_predict = pd.Series(model_3.predict(features_good_geo_3_valid))

In [49]:
mean_geo_3 =model_3_predict.mean()

In [50]:
display(mean_geo_3)

94.96504596800489

In [51]:
mse_3 = mean_squared_error(target_good_geo_3_valid,model_3_predict)
rmse_3 = mse_3**0.5
print(rmse_3)

40.02970873393434


Наименьший показатель RMSE был на 2 модели.
Проверим зависит это от датасета или модель обучена иначе, проверим модель 2, на датасете для 3 зоны.

In [52]:
model_2_predict_area_3 =pd.Series(model_2.predict(features_good_geo_3_valid))

In [53]:
mse_3_2 = mean_squared_error(target_good_geo_3_valid,model_2_predict_area_3)
rmse_3_2 = mse_3**0.5
print(rmse_3_2)

40.02970873393434


Показатель не изменился, модели обучены одинаково, дело в исходных данных.

# 3. Подготовка к расчёту прибыли


Создадим переменные с условиями задачи.

In [54]:
budget = 10000000000
price = 450000
location = 500
location_top = 200



Рассчитаем минимальный запас на скважине, для безубыточной добычи.

In [55]:
min_product = budget/price/location_top

Выведем наши данные:

In [56]:
print('Минимальный профит=',min_product)
print('Среднее по запасам зона 1 = ',mean_geo_1)
print('Среднее по запасам зона 2 = ',mean_geo_2)
print('Среднее по запасам зона 3 = ',mean_geo_3)

Минимальный профит= 111.11111111111111
Среднее по запасам зона 1 =  92.59256778438035
Среднее по запасам зона 2 =  68.72854689544602
Среднее по запасам зона 3 =  94.96504596800489


Средний запас ниже необходимого, для безубыточной добычи надо тщательно отбирать места для скважин.

# 4. Расчёт прибыли и рисков 

Напишем функцию, для расчёта прибыли.

In [57]:
def revenue(probabilities, real, stock):
    probs_sorted = probabilities.sort_values(ascending=False)
    sort = probs_sorted[:stock]
    selected = real.reset_index(drop=True)[sort.index]
    product = selected.sum()
    return product*price-budget

Расчитаем прибыль для наших регионов

In [58]:
revenue(model_1_predict,target_good_geo_1_valid,location_top)

3320826043.1398525

In [59]:
revenue(model_2_predict,target_good_geo_2_valid,location_top)

2415086696.681511

In [60]:
revenue(model_3_predict,target_good_geo_3_valid,location_top)

2710349963.5998325

Самым выгодным оказался первый регион.

### Проведём исследование выборок методом bootstrap 

Задаем рандом выборки семпл как константу.

In [61]:
state = RandomState(12345)


Создаем список для хранения информации выручке.

In [62]:
revenue_1 = []

Пишем цикл bootstrap

In [63]:
for i in range(1000):
    sub_probabilities = model_1_predict.sample(n=location, replace=True, random_state=state)
    sub_target = target_good_geo_1_valid
    rev = revenue(sub_probabilities, sub_target, location_top)
    revenue_1.append(rev)

In [64]:
revenue_1 = pd.Series(revenue_1)

Выводим необходимые данные для выбора оптимального региона по добыче.

In [65]:
print('Средняя прибыль=',revenue_1.mean())
print('95% доверительный интервал:   ','lower =',np.quantile(revenue_1,0.025),'upper= ',np.quantile(revenue_1,0.975))
print('Риск=',(revenue_1[revenue_1<0]).count()/1000*100,'%')

Средняя прибыль= 396164984.8023711
95% доверительный интервал:    lower = -111215545.89049526 upper=  909766941.5534225
Риск= 6.9 %


Аналогично первому региону.

In [66]:
revenue_2 = []

In [67]:
for i in range(1000):
    sub_probabilities = model_2_predict.sample(n=location, replace=True, random_state=state)
    sub_target = target_good_geo_2_valid
    rev = revenue(sub_probabilities, sub_target, location_top)
    revenue_2.append(rev)

In [68]:
revenue_2 = pd.Series(revenue_2)

In [69]:
print('Средняя прибыль=',revenue_2.mean())
print('95% доверительный интервал:   ','lower =',np.quantile(revenue_2,0.025),'upper= ',np.quantile(revenue_2,0.975))
print('Риск=',(revenue_2[revenue_2<0]).count()/1000*100,'%')

Средняя прибыль= 461155817.27723974
95% доверительный интервал:    lower = 78050810.7517417 upper=  862952060.2637235
Риск= 0.7000000000000001 %


In [70]:
revenue_3 = []

In [71]:
for i in range(1000):
    sub_probabilities = model_3_predict.sample(n=location, replace=True, random_state=state)
    sub_target = target_good_geo_3_valid
    rev = revenue(sub_probabilities, sub_target, location_top)
    revenue_3.append(rev)

In [72]:
revenue_3 = pd.Series(revenue_3)

In [73]:
print('Средняя прибыль=',revenue_3.mean())
print('95% доверительный интервал:   ','lower =',np.quantile(revenue_3,0.025),'upper= ',np.quantile(revenue_3,0.975))
print('Риск=',(revenue_3[revenue_3<0]).count()/1000*100,'%')

Средняя прибыль= 392950475.17060447
95% доверительный интервал:    lower = -112227625.37857565 upper=  934562914.5511636
Риск= 6.5 %


**Вывод**

Наиболее благоприятный регион для добычи - регион номер 2, не смотря что средний запас во 1 регионе лучше, если бы у нас не было целевого признака добыча там рискованна.
Во втором регионе минимальное значение RMSE и дальнейшие расчёты показали самый благоприятный результат по средней прибыли ,

